In [119]:
from news_analysis import NewsAnalysis
na = NewsAnalysis('voxtest.jsonl', 'jezebeltest.jsonl')

In [144]:
corpus = na.corpus
corpus.columns = ['author', 'date', 'text', 'title', 'url', 'id', 'org', 'topic0', 'topic1', 'topic2', 'topic3', 'topic4', 'category']

In [146]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
tfidf = TfidfVectorizer()
tsvd = TruncatedSVD(n_components=900, n_iter=100)
tfs = tfidf.fit_transform(corpus.text.apply(lambda x: ' '.join(x)))
lsa = tsvd.fit_transform(tfs)
print(tsvd.explained_variance_ratio_.sum())

0.742510421884


In [154]:
from sklearn.naive_bayes import GaussianNB
clf = GaussianNB()
clf.fit(lsa, corpus['org'])
probs = pd.DataFrame(clf.predict_proba(lsa))

In [167]:
import json
json.dumps([1,3,4])

'[1, 3, 4]'